# Loading the Data

In [1]:
import tensorflow_datasets as tfds

In [2]:
train_ds, test_ds = tfds.load('cifar10', split=['train','test'], as_supervised=True, batch_size = -1)

2023-11-04 22:37:54.085795: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-04 22:37:54.146004: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-04 22:37:54.146054: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-04 22:37:54.146106: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-04 22:37:54.156853: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-04 22:37:54.157503: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [12]:
train_images, train_labels = tfds.as_numpy(train_ds)
test_images, test_labels = tfds.as_numpy(test_ds)

# Applying LBP on the Training Images

In [4]:
import numpy as np

In [5]:
def get_pixel_value(img, center, x, y):
    try:
        if (img[x, y] >= center).all():
            return 1
        else:
            return 0
    except IndexError:
        return 0

In [6]:
def calculate_lbp_value(img, x, y):
    center = img[x, y]
    binary_values = []
    
    binary_values.append(get_pixel_value(img, center, x-1, y-1))
    binary_values.append(get_pixel_value(img, center, x-1, y))
    binary_values.append(get_pixel_value(img, center, x-1, y+1))
    binary_values.append(get_pixel_value(img, center, x, y+1))
    binary_values.append(get_pixel_value(img, center, x+1, y+1))
    binary_values.append(get_pixel_value(img, center, x+1, y))
    binary_values.append(get_pixel_value(img, center, x+1, y-1))
    binary_values.append(get_pixel_value(img, center, x, y-1))
    
    lbp_value = 0
    for i in range(8):
        lbp_value += binary_values[i] * (2**i)
    
    return lbp_value


In [7]:
def calculate_lbp_image(img):
    height, width, _ = img.shape
    lbp_img = np.zeros((height, width), dtype=np.uint8)
    
    for x in range(1, height-1):
        for y in range(1, width-1):
            lbp_img[x, y] = calculate_lbp_value(img, x, y)
    
    return lbp_img

In [9]:
print(len(test_images))

10000


In [8]:
train_images_lbp = []
for i in range(0, len(train_images)):
    train_images_lbp.append(calculate_lbp_image(train_images[i]))

test_images_lbp = []
for a in range(0, len(test_images)):
    test_images_lbp.append(calculate_lbp_image(test_images[a]))

test_images_lbp = np.array(test_images_lbp)
train_images_lbp = np.array(train_images_lbp)

IndexError: index 49999 is out of bounds for axis 0 with size 10000

# Extracting SIFT Features

In [ ]:
import cv2
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import numpy as np


In [ ]:
def extract_sift_features(images_list):
    image_descriptor = []
    sift = cv2.xfeatures2d.SIFT_create()
    for image in images_list:
        _, descriptor = sift.detectAndCompute(image, None)
        image_descriptor.append(descriptor)
    
    return image_descriptor

In [ ]:
def kmean_bow(all_descrip, num_clusters):
    bow_dict = []
    kmeans = KMeans(n_clusters = num_clusters)
    kmeans.fit(all_descrip)
    bow_dict = kmeans.cluster_centers_
    return bow_dict

In [ ]:
def create_feature_bow(image_desc, BoW, num_cluster):
    X_features = []
    for i in range(len(image_desc)):
        features = np.array([0] * num_cluster)
        if image_desc[i] is not None:
            distance = cdist(image_desc[i], BoW)
            argmin = np.argmin(distance, axis = 1)
            for j in argmin:
                features[j] += 1
        X_features.append(features)
    
    return X_features

*Applying SIFT on Train Images and Creating Bag of Words*

In [ ]:
image_descriptors = extract_sift_features(train_images_lbp)
all_descriptors = []
for desciptor in image_descriptors:
    if desciptor is not None:
        for des in desciptor:
            all_descriptors.append(des)

num_cluster = 60
BoW = kmean_bow(all_descriptors, num_cluster)
X_features = create_feature_bow(image_descriptors, BoW, num_cluster)

[ WARN:0@162.205] global shadow_sift.hpp:13 SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736
/home/aarvee/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


*Applying SIFT on Test Images and Creating Bag of Words*

In [ ]:
image_descriptors = extract_sift_features(test_images_lbp)
all_descriptors = []
for descriptor in image_descriptors:
    if descriptor is not None:
        for des in descriptor:
            all_descriptors.append(des)

num_cluster = 60
BoW = kmean_bow(all_descriptors, num_cluster)
Y_features = create_feature_bow(image_descriptors, BoW, num_cluster)

# Implementing SVM on the Data

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
model_svm = SVC(C = 30, random_state=42)
model_svm.fit(X_features, Y_features)
pred_images = model_svm.predict(test_images)
print("Confustion Report: ")
print(classification_report(pred_images, test_labels))

# Plotting the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

conf_mat = confusion_matrix(test_images, pred_images)
sns.heatmap(conf_mat, annot = True)

*Extracting HoG Features*


In [ ]:
from skimage.feature import hog

def extract_hog_features(X):
    features = []
    for image in X:
        # Convert image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        # Ensure the image has the correct data type (CV_8U)
        gray_image = np.uint8(gray_image * 255)

        # Calculate HOG features
        fd = hog(gray_image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
        features.append(fd)
    return np.array(features)

In [ ]:
X_train_hog = extract_hog_features(train_images)
y_test_hog = extract_hog_features(test_images)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_hog = scaler.fit_transform(X_train_hog)
y_test_hog = scaler.transform(y_test_hog)

In [ ]:
clf_hog = SVC(C = 30, random_state=42)
clf_hog.fit(X_train_hog, train_labels)

pred_hog = clf_hog.predict(y_test_hog)
print(classification_report(test_labels, pred_hog))

# Plotting the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

conf_mat = confusion_matrix(test_labels, pred_hog)
sns.heatmap(conf_mat, annot = True)